In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.animation import FuncAnimation
import matplotlib.patches as ptch
from IPython.display import HTML
from scipy.optimize import linprog

In [2]:
plt.rcParams["figure.figsize"] = [12,12]
#If you have problems with latex at matplotlib just comment next two lines, this might help
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')
def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [3]:
def animate_trajectory(dim, traj, argmin):
    if (dim != 2):
        raise RuntimeError('Can\'t animate not in 2 dimension\.')
    fig, ax = plt.subplots()
    n = len(traj)
    def step(t):
        ax.cla()
        ax.plot([argmin[0]], [argmin[1]], 'o', color='green')
        #Level contours
        delta = 0.025
#         x = np.arange(-3, 3, delta)
#         y = np.arange(-3, 3, delta)
#         X, Y = np.meshgrid(x, y)
#         Z = np.zeros_like(X)
        domain = ptch.Rectangle((0, 0), 1, 1, linewidth=2, edgecolor='black', facecolor='none')
        ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], color='black')
        ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], 'o', color='black')
        ax.add_patch(domain)
        fix_scaling(ax)
        ax.axis('off')
    plt.close()
    return FuncAnimation(fig, step,
                     frames=range(n), interval=600)

In [4]:
n = 2
A = np.array([[1, 0],[0, 1]])
b = np.array([-1, 3])
c = 0

In [5]:
def f(x):
    return np.matmul(np.matmul(x.T, A), x) + np.matmul(b.T, x) + c
def f_grad(x):
    return 2 * np.matmul(A, x) + b
def f_hes(x):
    return 2 * A

# Projecting gradient descent

In [6]:
#Projecting gradient descent
start_point = np.zeros(n) + 0.5
traj = [list(start_point)]
eigenvals = np.linalg.eig(f_hes(start_point))
argmin = [0.5, -1.5]
α = 2 / np.sum(eigenvals[0])
x = start_point
for i in range(0,10):
    x = x - α * f_grad(x)
    traj.append(list(x))
    for i in range(n):
        x[i] = max(min(1, x[i]), 0)
    traj.append(list(x))
base_animation = animate_trajectory(n, traj, argmin)
HTML(base_animation.to_jshtml())  

# Inner point method

In [7]:
def F(x, t):
    return t * f(x) - sum(np.log(-np.matmul(A_S, x) + b_S))
def F_grad(x, t):
    temp = []
    for ai, bi in zip(A_S,b_S):
        temp.append(-ai / (np.matmul(ai, x) - bi))
    return t * f_grad(x) + sum(temp)
def F_hes(x, t):
    temp = []
    for ai, bi in zip(A_S,b_S):
        temp.append(np.matmul(ai[np.newaxis].T, ai[np.newaxis]) / (np.matmul(ai, x) - bi) ** 2)
    return t * f_hes(x) + sum(temp)

In [11]:
c_f = np.zeros(n)
#Triangle: 
A_T = np.array([[0, -1], [1, -1], [-1, -1]])
b_T = np.array([0, -1, -1])
resT = linprog(c_f, A_T, b_T)
#Square:
A_S = np.array([[-1, 0], [0, -1], [1, 0], [0, 1]])
b_S = np.array([0, 0, 1, 1])
resS = linprog(c_f, A_S, b_S)
start = None
if not resS.success:
    print('Can\'t find inner point.')
else:
    if (0 in (np.matmul(A_S, resS.x) - b_S)):
        start = resS.x
        grad = sum(A_S[np.matmul(A_S, resS.x) == b_S])
        for i in range(0, 10):
            eps = 0.1 ** i
            x = start - eps * grad
            if ((np.matmul(A_S, x) - b_S) < 0).all():
                start = x
                print(x)
                break
    else:
        start = resS.x
        print(resS.x)

[0.1 0.1]


In [12]:
argmin = [0.5, -1.5]
x = start
alpha1 = 1.3
t = 1
trajS = [x]
for i in range(0,10):
    x = x - np.matmul(np.linalg.inv(F_hes(x, t)), F_grad(x, t))
    t = t * alpha1
    trajS.append(x)
base_animation = animate_trajectory(2, trajS, argmin)
HTML(base_animation.to_jshtml())  